In [1]:
import pandas as pd
import numpy as np
import re
import scipy.stats as stats
import lxml

## Reading in Cities df

In [2]:
# Reading in City, Region and Population Data
cities = pd.read_html('C:/Users/markg/Documents/Python/UofM Course/datasets/wikipedia_data.html')[1]
cities = cities.iloc[:-1,[0,3,5,6,7,8]]

# Removing square brackets in values for all the columns
for cols in cities.columns:
    cities[cols] = cities[cols].str.replace(r'(\[.*\s*\d+\])|(—)|(\s*)','', regex = True)
    
cities.rename(columns = {"Population (2016 est.)[8]":"Population"},inplace=True)
cities.head()

,Metropolitan area,Population,NFL,MLB,NBA,NHL
0,NewYorkCity,20153634,GiantsJets,YankeesMets,KnicksNets,RangersIslandersDevils
1,LosAngeles,13310447,RamsChargers,DodgersAngels,LakersClippers,KingsDucks
2,SanFranciscoBayArea,6657982,49ersRaiders,GiantsAthletics,Warriors,Sharks
3,Chicago,9512999,Bears,CubsWhiteSox,Bulls,Blackhawks
4,Dallas–FortWorth,7233323,Cowboys,Rangers,Mavericks,Stars


## get_teams()

In [3]:
def get_teams(df1 = cities, sport = 'NHL', dropcols = [2,4,6]):
    
        
    df1 = df1.copy()
    sport = sport
    n = dropcols
    
    # Combining teams and cities and metro area
    a = pd.DataFrame(df1[sport].str.split('([0-9A-Z]+[a-z]*)',
                                              regex = True, 
                                              expand = True)
                .drop(n, axis = 1))

    # Creating New Col for Area
    a['Metropolitan area'] = df1['Metropolitan area']

    # Pivoting Df to give area to each team, some areas had more than one team in their cell
    result = (pd.melt(a, id_vars = 'Metropolitan area')
                 .drop('variable', axis = 1)
                 .replace("",np.nan)
                 .replace("—",np.nan)
                 .dropna()
                 .rename({'value':'Team'}, axis = 1)
                 .reset_index(drop = True)
            )

    return result

## get_wl_ratio()

In [4]:
def get_wl_ratio(df1, df2, df3):
    '''
    df1 should contain Team, W and L. 
    df2 should be from get_teams(). 
    df3 should be cities
    '''
    df1 = df1.copy()
    df2 = df2.copy()
    df3 = df3.copy()
    
    # Merging tables to put in W and L data, then converting to int
   
    df1.Team = df1.Team.astype(str)
    df2.Team = df2.Team.astype(str)
    
    a = (pd.merge(df1, df2 , how = 'outer', on = 'Team')
                 .dropna()
                 .reset_index(drop = True)
            )
    a = a.astype({'W': 'int','L' : 'int'})

    # Definiting WL ratio and adding column
    ratio = a['W']/(a['L']+a['W'])
    a['WL_ratio'] = ratio
    
    # Grouping by region, then finding mean
    a = (a.groupby('Metropolitan area')['WL_ratio'].agg(np.nanmean))
    
    result = (pd.merge(a, df3, how = 'inner', 
                      on = 'Metropolitan area')
              .iloc[:,:3]
              .set_index('Metropolitan area')
              .astype({'Population':'int', 
                       'WL_ratio': 'float'}))
             
    
    return result


## get_stats()

In [6]:
def get_stats(df):
    df = df.copy()
    
    population_by_region = df['Population']
    win_loss_by_region = df['WL_ratio']

    return stats.pearsonr(population_by_region, win_loss_by_region)


## NHL

In [7]:
#Reading in NHL Data
nhl_df = pd.read_csv('C:/Users/markg/Documents/Python/UofM Course/datasets/nhl.csv')
nhl_df = nhl_df[nhl_df['year']==2018] # We only want 2018

nhl_df['Team'] = nhl_df['team'].str.replace(r'*','', regex = True)  # Removing * from names
nhl_df = nhl_df[['Team','W','L']]  # only want team name, w and l

# Remove annoying formatting text
droplist = []
for i in range(nhl_df.shape[0]):
    row=nhl_df.iloc[i]
    if row['Team']==row['W'] and row['L']==row['W']:
        droplist.append(i)

nhl_df=nhl_df.drop(droplist)

# Get Team name from Last Name
nhl_df['Team'] = nhl_df.Team.str.extract(r'\b(\w+)$')


nhl_df.head()

,Team,W,L
1,Lightning,54,23
2,Bruins,50,20
3,Leafs,49,26
4,Panthers,44,30
5,Wings,30,39


In [8]:
nhl_teams = get_teams(df1 = cities, 
                      sport = 'NHL', 
                      dropcols = [2,4,6])

nhl = get_wl_ratio(df1 = nhl_teams, 
                   df2 = nhl_df, 
                   df3 = cities)

nhl_stats = get_stats(nhl)
nhl_stats

(0.012486162921209923, 0.9497182859911781)

## NBA

In [9]:
# Creating W, L and Team df
nba_df=pd.read_csv("C:/Users/markg/Documents/Python/UofM Course/datasets/nba.csv")
nba_df = nba_df[nba_df['year'] == 2018]

nba_df['Team'] = (nba_df['team']
                  .str.replace(r'*','', regex = True)  # Removing * from names
                  .str.replace(r'[(][0-9]*[)]\Z','', regex = True)
                  .str.replace(u'\xa0', u'', regex = True)
                  )

#Selecting three columns
nba_df = nba_df[['Team','W','L']]

# Get Team name from Last Name
nba_df['Team'] = nba_df.Team.str.replace(r'[A-Za-z]+ ', '', regex = True)

nba_df.head()

,Team,W,L
0,Raptors,59,23
1,Celtics,55,27
2,76ers,52,30
3,Cavaliers,50,32
4,Pacers,48,34


In [10]:
nba_teams = get_teams(df1 = cities, 
                  sport = 'NBA', 
                  dropcols = [2])

nba = get_wl_ratio(df1 = nba_df,
                   df2 = nba_teams, 
                   df3 = cities)
nba_stats = get_stats(nba)
nba_stats

(-0.17657160252844623, 0.3687474160446294)

## MLB

In [11]:
mlb_df = pd.read_csv('C:/Users/markg/Documents/Python/UofM Course/datasets/mlb.csv')
mlb_df = mlb_df[mlb_df['year'] == 2018]

#Getting team names
mlb_df['Team'] = (mlb_df.team
                  .str.replace('\ Sox', 'Sox', regex = True)
                  .str.replace('\ Sox', 'Sox', regex = True)
                  .str.replace(r'[A-Za-z]+ ', '', regex = True)
                  .str.replace(r'St. ', '', regex = True))

# Filtering columns                  
mlb_df = mlb_df[['Team','W','L']]

mlb_df.Team.sort_values().unique()

array(['Angels', 'Astros', 'Athletics', 'Braves', 'Brewers', 'Cardinals',
       'Cubs', 'Diamondbacks', 'Dodgers', 'Giants', 'Indians', 'Jays',
       'Mariners', 'Marlins', 'Mets', 'Nationals', 'Orioles', 'Padres',
       'Phillies', 'Pirates', 'Rangers', 'Rays', 'RedSox', 'Reds',
       'Rockies', 'Royals', 'Tigers', 'Twins', 'WhiteSox', 'Yankees'],
      dtype=object)

In [16]:
mlb_teams = get_teams(df1 = cities, 
                  sport = 'MLB', 
                  dropcols = [2])

# Tidying teams, like WhiteSox and RedSoxs were both Sox
mlb_teams['Team'] = (mlb_teams.Team
                     .str.replace('White', 'WhiteSox', regex = True)
                     .str.replace('Red$', 'RedSox', regex = True)
                    )
# Removing Sox Rox
patterndel = '(\ASox)'
filter = mlb_teams['Team'].str.contains(patterndel)
mlb_teams = mlb_teams[~filter].reset_index(drop = True)

mlb_teams.head()

C:\Users\markg\AppData\Local\Temp\ipykernel_13400\3409674945.py:12: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  filter = mlb_teams['Team'].str.contains(patterndel)


,Metropolitan area,Team
0,NewYorkCity,Yankees
1,LosAngeles,Dodgers
2,SanFranciscoBayArea,Giants
3,Chicago,Cubs
4,Dallas–FortWorth,Rangers


In [13]:
mlb = get_wl_ratio(df1 = mlb_df,
                  df2 = mlb_teams, 
                  df3 = cities)
mlb_stats = get_stats(mlb)
mlb_stats

(0.15027698302669307, 0.46370703378875583)

## NFL

In [14]:
nfl_df = pd.read_csv('C:/Users/markg/Documents/Python/UofM Course/datasets/nfl.csv')
nfl_df = nfl_df[nfl_df['year'] == 2018]

#Remove annoying formatting text
droplist = []
for i in range(nfl_df.shape[0]):
    row=nfl_df.iloc[i]
    if row['team']==row['W'] and row['L']==row['W']:
        droplist.append(i)

nfl_df=nfl_df.drop(droplist)

# #Getting team names
nfl_df['Team'] = (nfl_df.team
                  .str.replace(r'[A-Za-z]+ ', '', regex = True)
                  .str.replace(r'[*+]*', '', regex = True)
                  
                 )
# Filtering columns                  
nfl_df = nfl_df[['Team','W','L']]

nfl_df.head()

,Team,W,L
1,Patriots,11,5
2,Dolphins,7,9
3,Bills,6,10
4,Jets,4,12
6,Ravens,10,6


In [26]:
nfl_teams = get_teams(df1 = cities, 
                  sport = 'NFL', 
                  dropcols = [2])

nfl = get_wl_ratio(df1 = nfl_df,
                  df2 = nfl_teams, 
                  df3 = cities)

nfl_stats = get_stats(nfl)
nfl_stats
nfl

,WL_ratio,Population
Metropolitan area,,
Atlanta,0.437500,5789700
Baltimore,0.625000,2798886
Boston,0.687500,4794447
Buffalo,0.375000,1132804
Charlotte,0.437500,2474314
Chicago,0.750000,9512999
Cincinnati,0.375000,2165139
Cleveland,0.466667,2055612
Dallas–FortWorth,0.625000,7233323


## Question 5

In [2]:
def df_tidy(df):
    df = df.copy()
    df = df.drop('Population', axis = 1)
    return df

[NFL, NBA, NHL, MLB] = [df_tidy(nfl), df_tidy(nba), df_tidy(nhl), df_tidy(mlb)]    


sports = ['NFL', 'NBA', 'NHL', 'MLB']
p_value = pd.DataFrame({k: np.nan for k in sports}, index = sports)

for i in sports:
     for j in sports:
        if i != j:
            merge = pd.merge(locals()[i], locals()[j], how = 'inner', on = 'Metropolitan area')
            p_value.loc[i,j] = stats.ttest_rel(merge['WL_ratio_x'], merge['WL_ratio_y'])[1]

p_value



NameError: name 'nfl' is not defined